# Pipeline

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')


In [2]:
import datetime as DT
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import make_scorer, precision_score, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, TargetEncoder

In [10]:
import prophet
from prophet import Prophet

In [86]:
import random

#

## Загрузка датасета

In [52]:
data = pd.read_csv('data1.csv')
data

,date,st_id,pr_sku_id,pr_sales_type_id,price,total_sales,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,sales_total,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,is_holiday
0,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,130d817d8b3f616e97c555ec57946aec,1,0
1,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,005c52de11abaf8e0a9714b24415ce34,0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,c20ad4d76fe97759aa27a0c99bff6710,fb2fcd534b0ff3bbed73cc51df620323,572494dd9d43f96629bddcbd4102f35f,1,0
2,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,0,435.0,1.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,0
3,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,191.5,6.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
4,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0120fa9d93da50a73a6f61838b49ff46,0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,a98ba70a6aaa1d3da549927a4ca79dfa,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124667,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe50ae64d08d4f8245aaabc55d1baf79,1,143.2,5.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,100699.0,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,b3e70d072fc648f6934c288e44f5c35e,1,0
2124668,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe5d18ae6650335830e4c1dbd9e6ddb9,0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,100699.0,aab3238922bcc25a6f606eb525ffdc56,3de2334a314a7a72721f1f74a6cb4cee,0f3abfd67959c079e7b3759440e7415c,17,0
2124669,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff1f1e5d2708809c03d03ff8b689030b,0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,100699.0,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,1507fc0175c1ff0cf4440105bc6f93e6,17,0
2124670,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,0,46.0,3.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,100699.0,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,0


In [90]:
columns_to_drop = ['st_city_id', 'st_division_code', 'price',
                    'st_type_format_id', 'st_type_loc_id', 'st_type_size_id',
                      'st_is_active', 'sales_total', 'pr_group_id', 'pr_cat_id', 'pr_subcat_id', 'pr_uom_id']

In [92]:
data2 = data.drop(columns_to_drop, axis =1).copy()
data2 = data2.rename(columns={'date': 'ds', 'total_sales': 'y', 'is_holiday': 'holiday'}) 
data2['ds'] = pd.to_datetime(data2['ds'])
data2

,ds,st_id,pr_sku_id,pr_sales_type_id,y,holiday
0,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,0,0.0,0
1,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,005c52de11abaf8e0a9714b24415ce34,0,0.0,0
2,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,0,1.0,0
3,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0
4,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0120fa9d93da50a73a6f61838b49ff46,0,0.0,0
...,...,...,...,...,...,...
2124667,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe50ae64d08d4f8245aaabc55d1baf79,1,5.0,0
2124668,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe5d18ae6650335830e4c1dbd9e6ddb9,0,0.0,0
2124669,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff1f1e5d2708809c03d03ff8b689030b,0,0.0,0
2124670,2023-07-18,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,0,3.0,0


In [132]:
unique_segments = data2[['st_id', 'pr_sku_id']].drop_duplicates()
random_segments = unique_segments.sample(30)
pairs_list = random_segments.values.tolist()

In [133]:
pairs_list

[['16a5cdae362b8d27a1d8f8c7b78b4330', '67731b6c536b98e9886698772595a486'],
 ['f7e6c85504ce6e82442c770f7c8606f0', '63fd103c3678af110f183d7d499d0081'],
 ['6364d3f0f495b6ab9dcf8d3b5c6e0b01', 'd30eea686dd9375430858468d5aa8824'],
 ['c81e728d9d4c2f636f067f89cc14862c', 'bb675d274470402a269595bf932c364d'],
 ['6364d3f0f495b6ab9dcf8d3b5c6e0b01', '771d9ed245872c553b1472e9f43e1860'],
 ['6364d3f0f495b6ab9dcf8d3b5c6e0b01', '57e82787ed765d35b35a286918e7fb71'],
 ['1ecfb463472ec9115b10c292ef8bc986', '6ef8d0eb99348b6be4cf97f499d5e6c0'],
 ['f7e6c85504ce6e82442c770f7c8606f0', 'f5a455e5fea834c85429331d934c40fa'],
 ['fa7cdfad1a5aaf8370ebeda47a1ff1c3', '661f94d2119410f42182d58be5e312f9'],
 ['c81e728d9d4c2f636f067f89cc14862c', 'e3334f5df6c5bf053070749a5f70fbc5'],
 ['fa7cdfad1a5aaf8370ebeda47a1ff1c3', 'b688e55fb702f9522be5a2b536220ab5'],
 ['fa7cdfad1a5aaf8370ebeda47a1ff1c3', 'be9dd1e0dfe50a60a56015e221bd2033'],
 ['c81e728d9d4c2f636f067f89cc14862c', 'a08abd96f9e056ad8cf9db4ae5f5754a'],
 ['c81e728d9d4c2f636f067f

## Вспомогательные функции

In [119]:
def wape(y_true, y_pred):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))

In [121]:
def forecast_prophet(segment):
    train_data = segment.iloc[:-14]
    test_data = segment.iloc[-14:]

    model = Prophet()
    model.fit(train_data)

    future = model.make_future_dataframe(periods=14)

    forecast = model.predict(future)

    predicted_values = forecast.iloc[-14:]['yhat'].values

    true_values = test_data['y'].values

    wape_score = wape(true_values, predicted_values)
    print(f'WAPE for 14-day prediction: {wape_score}')
    #return forecast

## Делаем петлю

In [115]:
dfs = {}

In [134]:
for pair in pairs_list:
    st, sku = pair
    segment = data2[(data['st_id'] == st) & (data2['pr_sku_id'] == sku)]
    #dfs[f'{st}_{sku}'] = segment
    forecast_prophet(segment)

12:44:09 - cmdstanpy - INFO - Chain [1] start processing


12:44:09 - cmdstanpy - INFO - Chain [1] done processing
12:44:09 - cmdstanpy - INFO - Chain [1] start processing
12:44:09 - cmdstanpy - INFO - Chain [1] done processing


WAPE for 14-day prediction: 1.0413170385145363
WAPE for 14-day prediction: 1.5060757003216636


12:44:10 - cmdstanpy - INFO - Chain [1] start processing
12:44:10 - cmdstanpy - INFO - Chain [1] done processing
12:44:11 - cmdstanpy - INFO - Chain [1] start processing
12:44:11 - cmdstanpy - INFO - Chain [1] done processing


WAPE for 14-day prediction: 0.4629582822001877


...

WAPE for 14-day prediction: 0.6775610720311085


12:44:21 - cmdstanpy - INFO - Chain [1] start processing


WAPE for 14-day prediction: 0.5321942068069749


12:44:21 - cmdstanpy - INFO - Chain [1] done processing
12:44:22 - cmdstanpy - INFO - Chain [1] start processing
12:44:22 - cmdstanpy - INFO - Chain [1] done processing


WAPE for 14-day prediction: inf
WAPE for 14-day prediction: 0.6770851351256486


In [102]:
#dfs

## Все, что ниже - подготовительный этап, который требовался, чтобы найти оптимальный поход к созданию функции

In [53]:
segment = data[(data['st_id']=='16a5cdae362b8d27a1d8f8c7b78b4330') & (data['pr_sku_id']=='0094042bfeae507dc7f62acc8e5ed03a')]
display(segment)
print(segment[segment['total_sales']!=0]['price'].count())

,date,st_id,pr_sku_id,pr_sales_type_id,price,total_sales,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,sales_total,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,is_holiday
3,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,191.5,6.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
6039,2022-08-02,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,202.5,8.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
12075,2022-08-03,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,220.7,3.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
18111,2022-08-04,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,193.2,16.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
24147,2022-08-05,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,199.5,27.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094495,2023-07-14,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,170.6,18.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
2100531,2023-07-15,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,210.9,13.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
2106567,2023-07-16,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0
2112603,2023-07-17,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,198.4,6.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,143898.0,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,0


308


In [54]:
columns_to_drop = ['st_city_id', 'st_division_code', 'price',
                    'st_type_format_id', 'st_type_loc_id', 'st_type_size_id',
                      'st_is_active', 'sales_total', 'pr_group_id', 'pr_cat_id', 'pr_subcat_id', 'pr_uom_id']

In [55]:
segment = segment.drop(columns_to_drop, axis =1)
segment

,date,st_id,pr_sku_id,pr_sales_type_id,total_sales,is_holiday
3,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0
6039,2022-08-02,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,8.0,0
12075,2022-08-03,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,3.0,0
18111,2022-08-04,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,16.0,0
24147,2022-08-05,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,27.0,0
...,...,...,...,...,...,...
2094495,2023-07-14,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,18.0,0
2100531,2023-07-15,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,13.0,0
2106567,2023-07-16,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,0.0,0
2112603,2023-07-17,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0


In [56]:
segment = segment.rename(columns={'date': 'ds', 'total_sales': 'y', 'is_holiday': 'holiday'}) 

In [57]:
segment['ds'] = pd.to_datetime(segment['ds'])

In [45]:
#segment = segment.reset_index(inplace = True)

In [58]:
segment.dtypes

ds                  datetime64[ns]
st_id                       object
pr_sku_id                   object
pr_sales_type_id             int64
y                          float64
holiday                      int64
dtype: object

In [59]:
segment

,ds,st_id,pr_sku_id,pr_sales_type_id,y,holiday
3,2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0
6039,2022-08-02,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,8.0,0
12075,2022-08-03,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,3.0,0
18111,2022-08-04,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,16.0,0
24147,2022-08-05,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,27.0,0
...,...,...,...,...,...,...
2094495,2023-07-14,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,18.0,0
2100531,2023-07-15,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,13.0,0
2106567,2023-07-16,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,0.0,0
2112603,2023-07-17,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0


Ну ладненько, образец взяли. Пропусков за год там немного. Теперь попробуем пайплайн построить только для него. 

## Сразу делим на трейн и тест, чтобы предотвратить утечку целевого признака. А она точно есть, как минимум в средней скользящей. А еще она есть в 

In [61]:
train = segment[segment.ds <= '2023-07-04']
test = segment[segment.ds > '2023-07-04']
test 

,ds,st_id,pr_sku_id,pr_sales_type_id,y,holiday
2040171,2023-07-05,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,5.0,0
2046207,2023-07-06,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,22.0,0
2052243,2023-07-07,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,23.0,0
2058279,2023-07-08,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,28.0,0
2064315,2023-07-09,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,18.0,0
2070351,2023-07-10,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,7.0,0
2076387,2023-07-11,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,8.0,0
2082423,2023-07-12,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,11.0,0
2088459,2023-07-13,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,15.0,0
2094495,2023-07-14,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,18.0,0


In [62]:
# нашел интересный пайплайн/фунцию. Пускай будет под рукой

In [63]:
def forecast_prophet(segment_data):
    model = Prophet()
    model.fit(segment_data)
    future = model.make_future_dataframe(periods=len(test_data))
    forecast = model.predict(future)
    return forecast

In [64]:
model = Prophet()

In [65]:
model.fit(train)

09:53:38 - cmdstanpy - INFO - Chain [1] start processing
09:53:39 - cmdstanpy - INFO - Chain [1] done processing


In [66]:
period = 14
future = model.make_future_dataframe(periods=period)
forecast = model.predict(future)

In [68]:
display(forecast[-14:])

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
338,2023-07-05,15.201410,-0.705106,21.536294,15.201410,15.201410,-4.988364,-4.988364,-4.988364,-4.988364,-4.988364,-4.988364,0.0,0.0,0.0,10.213047
339,2023-07-06,15.252033,0.064108,23.895807,15.252033,15.252033,-3.176613,-3.176613,-3.176613,-3.176613,-3.176613,-3.176613,0.0,0.0,0.0,12.075421
340,2023-07-07,15.302656,8.686900,31.043868,15.302646,15.302752,4.314917,4.314917,4.314917,4.314917,4.314917,4.314917,0.0,0.0,0.0,19.617573
341,2023-07-08,15.353279,13.449667,36.688638,15.352458,15.354314,10.308122,10.308122,10.308122,10.308122,10.308122,10.308122,0.0,0.0,0.0,25.661401
342,2023-07-09,15.403902,3.876235,27.492690,15.402038,15.406314,0.572298,0.572298,0.572298,0.572298,0.572298,0.572298,0.0,0.0,0.0,15.976201
343,2023-07-10,15.454525,1.613230,24.668993,15.451087,15.458195,-2.384722,-2.384722,-2.384722,-2.384722,-2.384722,-2.384722,0.0,0.0,0.0,13.069803
344,2023-07-11,15.505148,-0.977709,23.116327,15.499997,15.510753,-4.645639,-4.645639,-4.645639,-4.645639,-4.645639,-4.645639,0.0,0.0,0.0,10.859509
345,2023-07-12,15.555771,-0.962630,21.909423,15.548882,15.563560,-4.988364,-4.988364,-4.988364,-4.988364,-4.988364,-4.988364,0.0,0.0,0.0,10.567407
346,2023-07-13,15.606394,0.949079,24.137501,15.597357,15.616724,-3.176613,-3.176613,-3.176613,-3.176613,-3.176613,-3.176613,0.0,0.0,0.0,12.429781
347,2023-07-14,15.657017,7.979451,31.755608,15.645649,15.669950,4.314917,4.314917,4.314917,4.314917,4.314917,4.314917,0.0,0.0,0.0,19.971933


In [103]:
#model.plot(forecast)

In [72]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(segment.set_index('ds'))
cmp_df

,yhat,yhat_lower,yhat_upper,st_id,pr_sku_id,pr_sales_type_id,y,holiday
ds,,,,,,,,
2022-08-01,6.178670,-5.670245,17.907944,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,6.0,0
2022-08-02,3.907170,-8.505790,15.979227,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,8.0,0
2022-08-03,3.553861,-7.264373,15.035401,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,3.0,0
2022-08-04,5.355029,-5.867414,17.311224,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,16.0,0
2022-08-05,12.835974,2.037839,24.257527,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,27.0,0
...,...,...,...,...,...,...,...,...
2023-07-14,19.971933,7.979451,31.755608,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,1,18.0,0
2023-07-15,26.015762,14.218969,37.046731,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,13.0,0
2023-07-16,16.330561,4.044679,28.153635,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,0,0.0,0


In [75]:
def wape(y_true, y_pred):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))

In [76]:
print(wape(cmp_df[-period:]['y'], cmp_df[-period:]['yhat']))

0.43761590868101763
